# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# store filepath in a variable
data_file ="../output_data/cities_data.csv"

# read data file with the pandas library
cities_data_df = pd.read_csv(data_file)




In [3]:
# show the first five lines 
cities_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Hofn,64.2539,-15.2082,42.01,91,100,8.70,IS,1619372838
1,Bluff,-46.6000,168.3333,42.01,84,79,5.99,NZ,1619372838
2,Lata,40.1629,-8.3327,62.01,89,90,3.20,PT,1619372839
3,Evanston,42.0411,-87.6901,46.99,46,20,11.50,US,1619372667
4,Mehamn,71.0357,27.8492,35.60,69,75,11.50,NO,1619372839


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Store latitude and longitude in locations
locations = cities_data_df[["Lat", "Lng"]]

# Humidity as heat map weight
rating = cities_data_df["Humidity"].astype(float)

# create heat map maximum_intensity variable
maximum_intensity = cities_data_df["Humidity"].max().astype(float)
maximum_intensity



100.0

In [5]:
rating

0      91.0
1      84.0
2      89.0
3      46.0
4      69.0
       ... 
586    89.0
587    73.0
588    89.0
589    89.0
590    61.0
Name: Humidity, Length: 591, dtype: float64

In [6]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)


In [7]:
# Plot Heatmap
fig = gmaps.figure()

# Create heatmap layer
heatmap_layer = gmaps.heatmap_layer(locations, weights=rating)

# Add layer
fig.add_layer(heatmap_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# configure heatlayer attributes 
heatmap_layer.max_intensity = maximum_intensity
#heat_layer.point_radius = 1
#heat_layer.dissipating = False


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
# create new dataframe fitting "ideal" weather criteria
# criteria: max temp 75-90 F, < 40% Humidity, wind speed < 10

ideal_weather_df = cities_data_df.copy()
# filter by temp
ideal_weather_df = ideal_weather_df[(ideal_weather_df['Max Temp'] > 75)]
ideal_weather_df = ideal_weather_df[(ideal_weather_df['Max Temp'] < 90)]
# filter humidity
ideal_weather_df = ideal_weather_df[(ideal_weather_df['Humidity'] < 40)]
# filter wind speed
ideal_weather_df = ideal_weather_df[(ideal_weather_df['Wind Speed'] < 10)]
#filter cloudiness
ideal_weather_df = ideal_weather_df[(ideal_weather_df['Cloudiness'] < 10)]
ideal_weather_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
20,Upington,-28.4478,21.2561,77.00,19,0,3.44,ZA,1619372844
77,Estación Coahuila,32.1939,-115.0017,81.00,18,1,0.98,MX,1619372860
220,Río Frío de Juárez,19.3500,-98.6667,77.00,25,1,5.75,MX,1619372900
372,Vallenar,-28.5708,-70.7581,79.86,29,0,6.22,CL,1619372945
448,Novo Cruzeiro,-17.4681,-41.8753,85.05,38,5,0.27,BR,1619372967
456,Sonoita,31.8500,-112.8333,82.00,15,1,3.00,MX,1619372723
488,Gumdag,39.2061,54.5906,76.80,35,6,8.08,TM,1619372977


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
# store ideal weather df into new variable 
hotel_df = ideal_weather_df

# add "hotel name" column
hotel_df['Hotel Name'] = ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
20,Upington,-28.4478,21.2561,77.00,19,0,3.44,ZA,1619372844,
77,Estación Coahuila,32.1939,-115.0017,81.00,18,1,0.98,MX,1619372860,
220,Río Frío de Juárez,19.3500,-98.6667,77.00,25,1,5.75,MX,1619372900,
372,Vallenar,-28.5708,-70.7581,79.86,29,0,6.22,CL,1619372945,
448,Novo Cruzeiro,-17.4681,-41.8753,85.05,38,5,0.27,BR,1619372967,
456,Sonoita,31.8500,-112.8333,82.00,15,1,3.00,MX,1619372723,
488,Gumdag,39.2061,54.5906,76.80,35,6,8.08,TM,1619372977,


In [11]:
hotel_df = hotel_df.reset_index(drop=True)
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Upington,-28.4478,21.2561,77.00,19,0,3.44,ZA,1619372844,
1,Estación Coahuila,32.1939,-115.0017,81.00,18,1,0.98,MX,1619372860,
2,Río Frío de Juárez,19.3500,-98.6667,77.00,25,1,5.75,MX,1619372900,
3,Vallenar,-28.5708,-70.7581,79.86,29,0,6.22,CL,1619372945,
4,Novo Cruzeiro,-17.4681,-41.8753,85.05,38,5,0.27,BR,1619372967,
5,Sonoita,31.8500,-112.8333,82.00,15,1,3.00,MX,1619372723,
6,Gumdag,39.2061,54.5906,76.80,35,6,8.08,TM,1619372977,


In [12]:
# set parameters to search for hotels within 5000 meters
params = {
    "radius": 5000,
    "types": "lodging",
    "Keyword": "hotel",
    "key": g_key
}

# use the lat/lng to identify lodging
for index, row in hotel_df.iterrows():
    # get lat, lng from hotel_df
    lat = row["Lat"]
    lng = row["Lng"]
    
    #change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
    # convert to json
    hotel_name = hotel_name.json()
#    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
       hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
#        hotel_df.loc[index, "Hotel Address"] = name_address["results"][0]["vicinity"]
#        hotel_df.loc[index, "Hotel Rating"] = name_address["results"][0]["rating"]
    except (KeyError, IndexError):
        print("No hotel found... skipping.")
    
    
    
    

No hotel found... skipping.


In [13]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Upington,-28.4478,21.2561,77.00,19,0,3.44,ZA,1619372844,River Place Manor
1,Estación Coahuila,32.1939,-115.0017,81.00,18,1,0.98,MX,1619372860,Apartamentos Julissa
2,Río Frío de Juárez,19.3500,-98.6667,77.00,25,1,5.75,MX,1619372900,Rancho el guarda
3,Vallenar,-28.5708,-70.7581,79.86,29,0,6.22,CL,1619372945,Humacao Bed & Breakfast
4,Novo Cruzeiro,-17.4681,-41.8753,85.05,38,5,0.27,BR,1619372967,Hotel Nossa Senhora das Graças
5,Sonoita,31.8500,-112.8333,82.00,15,1,3.00,MX,1619372723,Hotel Nora
6,Gumdag,39.2061,54.5906,76.80,35,6,8.08,TM,1619372977,


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations,
    info_box_content=hotel_info)

fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))